# Atlantic Migratory Fish Habitat Priority

This indicator was used in 2022 Base Blueprint.

It was modified from the 2021 SA indicator by change the extent to cover a larger area

Created by Amy Keister, last run by Amy Keister 23 February, 2022


In [6]:
import os
import arcpy

In [ ]:
import time
start = time.time()

In [7]:
# define spatial reference and workspaces
sr = arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"E:\WORKING\BaseBlueprint\2022\3_Indicators\AtlanticMigratoryFishHabitat\AtlanticMigratoryFishHabitat.gdb"

In [8]:
# define final indicator outputs
SEout = r"E:\WORKING\BaseBlueprint\2022\3_Indicators\AtlanticMigratoryFishHabitat.tif"

In [9]:
# define rasters used for cell size, extent, and snapping
SEraster= r"E:\WORKING\BaseBlueprint\2022\2_Subregions\BaseBlueprintExtent2022.tif"

In [10]:
# define inputs
# species layers from the SEACAP project
sa= r"F:\GIS_DATA\DecisionSupportTools\ACFHP_FishHabitatConservationAreaMapping\South Atlantic Diadromous Analysis\data\v101\diadromous_northern_results.gdb\Northern_Diadromous_Catchments_Corrected_05302019"
ma= r"F:\GIS_DATA\DecisionSupportTools\ACFHP_FishHabitatConservationAreaMapping\Mid-Atlantic Diadromous Analysis\data\v101\finallayers.gdb\catchments_Mid_Atlantic"
# EPA floodplain
FP= r"F:\GIS_DATA\WaterResources\Estimated_floodplain_CONUS\Estimated_floodplain_CONUS.tif"

## Start Analysis

In [6]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [7]:
print(arcpy.env.workspace)

E:\WORKING\BaseBlueprint\2022\3_Indicators\AtlanticMigratoryFishHabitat\AtlanticMigratoryFishHabitat.gdb


### bring in ACFHP data

In [8]:
# make a copy of the south atlantic layer for edits
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures(sa, "sa", '', None, None, None)

In [9]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster):
    arcpy.conversion.FeatureToRaster(in_features="sa", field="FINALSCORE", out_raster="saR", cell_size=SEraster)

In [10]:
# make a copy of the mid atlantic layer for edits
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures(ma, "ma", '', None, None, None)

In [11]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster):
    arcpy.conversion.FeatureToRaster(in_features="ma", field="TotalPoints", out_raster="maR", cell_size=SEraster)

In [12]:
# mosaic sa and ma rasters together
#arcpy.management.MosaicToNewRaster("saR;maR", OutWorkspace, "sama", SAraster, "8_BIT_UNSIGNED", None, 1, "FIRST", "FIRST")

In [13]:
# mosaic to new raster is creating an output with bands of missing data switching to a new method to combine
# use cell statistics to combine the raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster):
    out_raster = arcpy.sa.CellStatistics("saR;maR", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("saRmaR")

In [14]:
# reclassify
out_raster = arcpy.sa.Reclassify("saRmaR", "Value", "0 0;10 1;20 2;30 3;40 4;50 5;60 6;70 7;80 8", "DATA"); out_raster.save(r"saRmaRrclss")

### clip to floodplains

In [15]:
# Limit the result to the EPA floodplain
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster):
    out_raster = arcpy.sa.Con(FP, "saRmaRrclss", '',"Value = 1"); out_raster.save("indicator")

In [16]:
# If we want places outside the floodplain to be 0, we can use this one
# out_raster = arcpy.sa.Con(FP, "indicator2", '0',"Value = 1"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [11]:
#clip to SE 2021 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask("indicator", SEraster); out_raster.save("SEMask")

In [12]:
#export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("SEMask", SEout, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [4]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 8:
        return '8 = Final score of 80 (areas of excellent fish habitat)'
    elif value == 7:
        return '7 = Final score of 70 (areas of excellent fish habitat)' 
    elif value == 6:
        return '6 = Final score of 60 (restoration opportunity areas)' 
    elif value == 5:
        return '5 = Final score of 50 (restoration opportunity areas)'
    elif value == 4:
        return '4 = Final score of 40 (restoration opportunity areas)' 
    elif value == 3:
        return '3 = Final score of 30 (restoration opportunity areas)' 
    elif value == 2:
        return '2 = Final score of 20 (restoration opportunity areas)'
    elif value == 1:
        return '1 = Final score of 10 (degraded areas of opportunity)' 
    elif value == 0:
        return '0 = Final score of 0 (degraded areas of opportunity)' 
"""

In [5]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(SEout, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\AtlanticMig\\AtlanticMigratoryFishHabitat.tif'>

In [6]:
# set code block for next step
codeblock = """
def Reclass1(Value):
    if Value == 8:
        return 2
    elif Value == 7:
        return 4
    elif Value == 6:
        return 5
    elif Value == 5:
        return 31
    elif Value == 4:
        return 77
    elif Value == 3:
        return 127
    elif Value == 2:
        return 168
    elif Value == 1:
        return 204
    elif Value == 0:
        return 229
    else:
        return 0

def Reclass2(Value):
    if Value == 8:
        return 56
    elif Value == 7:
        return 85
    elif Value == 6:
        return 105
    elif Value == 5:
        return 129
    elif Value == 4:
        return 153
    elif Value == 3:
        return 173
    elif Value == 2:
        return 190
    elif Value == 1:
        return 207
    elif Value == 0:
        return 225
    else:
        return 0

def Reclass3(Value):
    if Value == 8:
        return 88
    elif Value == 7:
        return 133
    elif Value == 6:
        return 165
    elif Value == 5:
        return 184
    elif Value == 4:
        return 198
    elif Value == 3:
        return 210
    elif Value == 2:
        return 220
    elif Value == 1:
        return 229
    elif Value == 0:
        return 239
    else:
        return 0
"""

In [7]:
# calculate Red field
arcpy.management.CalculateField(SEout, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(SEout, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(SEout, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\AtlanticMig\\AtlanticMigratoryFishHabitat.tif'>

In [ ]:
end = time.time()
print(end - start)